In [1]:
# Load library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math 
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
%matplotlib inline

## Import Data 

In [2]:
# Create URL
test_csv = "~/COMP30027_2021_Project2_datasets/recipe_test.csv"
train_csv = "~/COMP30027_2021_Project2_datasets/recipe_train.csv"

In [3]:
# Load Dataset 
test_df = pd.read_csv(test_csv )
train_df = pd.read_csv(train_csv)

In [4]:
# Load CountVec model for name 
import pickle
name_vectorizer = pickle.load(open("/Users/hesterlim/COMP30027_2021_Project2_datasets/recipe_text_features_countvec/train_name_countvectorizer.pkl", "rb"))
name_dict = name_vectorizer.vocabulary_

/anaconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.21.3 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [5]:
# Load CountVec model for steps
import pickle
step_vectorizer = pickle.load(open("/Users/hesterlim/COMP30027_2021_Project2_datasets/recipe_text_features_countvec/train_steps_countvectorizer.pkl", "rb"))
step_dict = step_vectorizer.vocabulary_

In [6]:
# Load CountVec model for ingredients
import pickle
ingr_vectorizer = pickle.load(open("/Users/hesterlim/COMP30027_2021_Project2_datasets/recipe_text_features_countvec/train_ingr_countvectorizer.pkl", "rb"))
ingr_dict = ingr_vectorizer.vocabulary_

In [7]:
# training set 
X_train = train_df[['n_steps', 'n_ingredients']]
y_train = train_df['duration_label']
X_test = test_df[['n_steps', 'n_ingredients']]

## CountVec Name

In [8]:
# Extract Features from name column using Count Vectorizer
from sklearn.feature_extraction.text import CountVectorizer
name_train = train_df['name'].values # Create an array of name for training set
name_test = test_df['name'].values # Create an array of name for testing set
X_train_name = name_vectorizer.fit_transform(name_train)
X_test_name = name_vectorizer.transform(name_test)

In [9]:
print(X_train_name.shape, X_test_name.shape)

(40000, 10892) (10000, 10892)


In [10]:
# Are there any documents in X_test whose values are all 0? Why might this happen?
print(len(X_test_name.sum(axis=1)==0))
# This is hypothetically possible - if every word in one of the test documents had never appeared in the training data.
# For long documents, this is exceedingly unlikely due to the appearance of grammatical "words" such as _the_, _is_, and so on.

10000


### Feature Selection

In [11]:
# Choose best attributes
# Find out what the best 10 features were for your name dataset, according to  𝜒2
from sklearn.feature_selection import SelectKBest, chi2

x2 = SelectKBest(chi2, k=10)
X_train_name_x2 = x2.fit_transform(X_train_name,y_train) # Create a sparse matrix for CountVectorizer
X_test_name_x2 = x2.transform(X_test_name)

for feat_num in x2.get_support(indices=True):
    print(name_vectorizer.get_feature_names()[feat_num])

beef
cake
casserole
cooker
crock
crockpot
pot
roast
salad
slow


In [12]:
X_train_name_x2

<40000x10 sparse matrix of type '<class 'numpy.int64'>'
	with 7495 stored elements in Compressed Sparse Row format>

These seem like words that could be relevant to trying to distinguish between duration classification where salad could be made relatively fast and beef could be made relatively slow

Perhaps suprisingly are words like slow, cooker, crock and crockpot which are indicative of the cooking utensils not the food itself (and perhaps not of the problem more generally). It's difficult to determine the rare/common distinction here, but it becomes a little clearer as we look further down the ranking.

In [13]:
# Do the same thing for Mutual Information, instead of  𝜒2 (note that you want the classification version, not the regression version).
from sklearn.feature_selection import mutual_info_classif
mi = SelectKBest(score_func=mutual_info_classif, k=10)
X_train_name_mi = mi.fit_transform(X_train_name,y_train)
X_test_name_mi = mi.transform(X_test_name)

print(X_train_name_mi.shape, X_test_name_mi.shape)

for feat_num in mi.get_support(indices=True):
    print(name_vectorizer.get_feature_names()[feat_num])

(40000, 10) (10000, 10)
beef
cake
casserole
chicken
cooker
crock
pot
roast
salad
slow


Here we see more evidence of MI choosing frequently-occuring features, such as slow.

## Modelling

Build a classifier on the training dataset, and evaluate its Accuracy on the test set. Consider k-NN, and perhaps Naive Bayes or Decision Trees

It’s likely that the dataset is still small enough that you can build a model on the entire feature set (after the CountVectorizer, but before the SelectKBest) without crashing your computer. How well do these models predict the test data, using all of the features?

How does this compare with 1000 features, or just the top 10 features?

Try some different values for the cut-off for SelectKBest — is it possible to improve upon the Accuracy observed for the models which use the entire feature set? Is this more true for some learners than others? Does your choice between χ 2 and Mutual Information make a difference?

In [14]:
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB

# Models to train on 
models = [GaussianNB(),
          MultinomialNB(),
          DecisionTreeClassifier(max_depth=1)]
#          KNeighborsClassifier(n_neighbors=1),
#          KNeighborsClassifier(n_neighbors=5),
#          DecisionTreeClassifier(max_depth=None)]
#          svm.LinearSVC(C=C),
#          svm.SVC(kernel='rbf', gamma=0.7, C=C),
#          svm.SVC(kernel='poly', degree=3, C=C)]

# Model Titles
titles = ['GNB',
          'MNB',
          'one-r']
#          '1-nearest neighbour',
#          '5-nearest neighbour',
#          'Decision Tree']
#          'LinearSVC',
#          'SVM with a cubic kernel',
#          'SVM with an RBF kernel']

# Select Number of Features
k = 1000

# Chi-square
x2 = SelectKBest(chi2, k=k)
x2.fit(X_train_name,y_train)
X_train_name_x2 = x2.transform(X_train_name)
X_test_name_x2 = x2.transform(X_test_name)

# Mutual Information 
mi = SelectKBest(score_func=mutual_info_classif, k=k)
mi.fit(X_train_name,y_train)
X_train_name_mi = mi.transform(X_train_name)
X_test_name_mi = mi.transform(X_test_name)

# Fit the model and test the model 
Xs = [(X_train_name, X_test_name), (X_train_name_x2, X_test_name_x2), (X_train_name_mi, X_test_name_mi)]
X_names = ['complete', 'x2', 'mi']
for title, model in zip(titles, models):
    print('\n',title, '(with k=',k,'features):')
    for X_name, X in zip(X_names, Xs):
        X_train_t, X_test_t = X
        # convert the variable into a matrix and train it with the selected model 
        model.fit(X_train_t.todense(), y_train)
        acc = model.score(X_train_t.todense(), y_train)
        print(X_name, 'acc',  acc)


 GNB (with k= 1000 features):
complete acc 0.419725
x2 acc 0.5769
mi acc 0.2231

 MNB (with k= 1000 features):
complete acc 0.756375
x2 acc 0.69515
mi acc 0.70625

 one-r (with k= 1000 features):
complete acc 0.5185
x2 acc 0.5185
mi acc 0.5185


Here, we could observe that Multinomial Naive Bayes is the best model and Mutual Information is the best feature extraction method for the feature name in the training set

## CountVec Steps

In [15]:
# Extract Features from steps column using Count Vectorizer
from sklearn.feature_extraction.text import CountVectorizer
step_train = train_df['steps'].values # Create an array of name for training set
step_test = test_df['steps'].values # Create an array of name for testing set
X_train_step = step_vectorizer.fit_transform(step_train)
X_test_step = step_vectorizer.transform(step_test)

In [16]:
print(X_train_step.shape, X_test_step.shape)

(40000, 17967) (10000, 17967)


### Feature Selection

In [17]:
# Choose best attributes
# Find out what the best 10 features were for your name dataset, according to  𝜒2
from sklearn.feature_selection import SelectKBest, chi2

x2 = SelectKBest(chi2, k=10)
X_train_step_x2 = x2.fit_transform(X_train_step,y_train) # Create a sparse matrix for CountVectorizer
X_test_step_x2 = x2.transform(X_test_step)

for feat_num in x2.get_support(indices=True):
    print(step_vectorizer.get_feature_names()[feat_num])

30
bake
baking
cooker
crock
crockpot
hours
minutes
oven
slow


The best attributes for step column is similar to name column. This might due to the fact that words that describe time such as hours, minutes, slow indicate specific duration labels 

In [18]:
# Do the same thing for Mutual Information, instead of  𝜒2 (note that you want the classification version, not the regression version).
from sklearn.feature_selection import mutual_info_classif
mi = SelectKBest(score_func=mutual_info_classif, k=10)
X_train_step_mi = mi.fit_transform(X_train_step,y_train)
X_test_step_mi = mi.transform(X_test_step)

print(X_train_step_mi.shape, X_test_step_mi.shape)

for feat_num in mi.get_support(indices=True):
    print(step_vectorizer.get_feature_names()[feat_num])

(40000, 10) (10000, 10)
30
350
45
bake
baking
crock
hours
minutes
oven
preheat


Here, we could see that MI provide more words that describe time such as hours, minutes, 30, 350 and 45. 

In [19]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB

# Models to train on 
models = [GaussianNB(),
          MultinomialNB(),
          DecisionTreeClassifier(max_depth=1)]
#          KNeighborsClassifier(n_neighbors=1),
#          KNeighborsClassifier(n_neighbors=5),
#          DecisionTreeClassifier(max_depth=None)]
#          svm.LinearSVC(C=C),
#          svm.SVC(kernel='rbf', gamma=0.7, C=C),
#          svm.SVC(kernel='poly', degree=3, C=C)]

# Model Titles
titles = ['GNB',
          'MNB',
          'one-r']
#          '1-nearest neighbour',
#          '5-nearest neighbour',
#          'Decision Tree']
#          'LinearSVC',
#          'SVM with a cubic kernel',
#          'SVM with an RBF kernel']

# Select Number of Features
k = 1000

# Chi-square
x2 = SelectKBest(chi2, k=k)
x2.fit(X_train_step,y_train)
X_train_step_x2 = x2.transform(X_train_step)
X_test_step_x2 = x2.transform(X_test_step)

# Mutual Information 
mi = SelectKBest(score_func=mutual_info_classif, k=k)
mi.fit(X_train_step,y_train)
X_train_step_mi = mi.transform(X_train_step)
X_test_step_mi = mi.transform(X_test_step)

# Chi-square -> (X_train_step_x2, X_test_step_x2)
# Mutual Information -> (X_train_step_mi, X_test_step_mi) 

# Fit the model and test the model 
Xs = [(X_train_step, X_test_step), (X_train_step_x2, X_test_step_x2), (X_train_step_mi, X_test_step_mi)]
X_names = ['complete', 'x2', 'mi']
for title, model in zip(titles, models):
    print('\n',title, '(with k=',k,'features):')
    for X_name, X in zip(X_names, Xs):
        X_train_t, X_test_t = X
        # convert the variable into a matrix and train it with the selected model 
        model.fit(X_train_t.todense(), y_train)
        acc = model.score(X_train_t.todense(), y_train)
        print(X_name, 'acc',  acc)


 GNB (with k= 1000 features):
complete acc 0.40335
x2 acc 0.650025
mi acc 0.5815

 MNB (with k= 1000 features):
complete acc 0.751825
x2 acc 0.711875
mi acc 0.716325

 one-r (with k= 1000 features):
complete acc 0.6481
x2 acc 0.6481
mi acc 0.6481


Here, we could observe that Multinomial Naive Bayes is the best model and Mutual Information is the best feature extraction method for the feature step in the training set

## CountVec Ingredients

In [20]:
# Extract Features from name column using Count Vectorizer
from sklearn.feature_extraction.text import CountVectorizer
ingr_train = train_df['ingredients'].values # Create an array of name for training set
ingr_test = test_df['ingredients'].values # Create an array of name for testing set
X_train_ingr = ingr_vectorizer.fit_transform(ingr_train)
X_test_ingr = ingr_vectorizer.transform(ingr_test)

In [21]:
print(X_train_ingr.shape, X_test_ingr.shape)

(40000, 2906) (10000, 2906)


### Feature Selection

In [22]:
# Choose best attributes
# Find out what the best 10 features were for your name dataset, according to  𝜒2
from sklearn.feature_selection import SelectKBest, chi2

x2 = SelectKBest(chi2, k=10)
X_train_ingr_x2 = x2.fit_transform(X_train_ingr,y_train) # Create a sparse matrix for CountVectorizer
X_test_ingr_x2 = x2.transform(X_test_ingr)

for feat_num in x2.get_support(indices=True):
    print(ingr_vectorizer.get_feature_names()[feat_num])

baking
beef
butter
chicken
eggs
flour
potatoes
roast
soup
stew


In the ingredients colum, we could observe that both the cooking materials and cooking methods are selected as among the best attributes

In [23]:
# Do the same thing for Mutual Information, instead of  𝜒2 (note that you want the classification version, not the regression version).
from sklearn.feature_selection import mutual_info_classif
mi = SelectKBest(score_func=mutual_info_classif, k=10)
X_train_ingr_mi = mi.fit_transform(X_train_ingr,y_train)
X_test_ingr_mi = mi.transform(X_test_ingr)

print(X_train_ingr_mi.shape, X_test_ingr_mi.shape)

for feat_num in mi.get_support(indices=True):
    print(ingr_vectorizer.get_feature_names()[feat_num])

(40000, 10) (10000, 10)
baking
beef
butter
eggs
flour
onion
potatoes
roast
salt
sugar


Similar to chi-square. Both cooking methods and cooking materials are among the best attribites

In [24]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB

# Models to train on 
models = [GaussianNB(),
          MultinomialNB(),
          DecisionTreeClassifier(max_depth=1)]
#          KNeighborsClassifier(n_neighbors=1),
#          KNeighborsClassifier(n_neighbors=5),
#          DecisionTreeClassifier(max_depth=None)]
#          svm.LinearSVC(C=C),
#          svm.SVC(kernel='rbf', gamma=0.7, C=C),
#          svm.SVC(kernel='poly', degree=3, C=C)]

# Model Titles
titles = ['GNB',
          'MNB',
          'one-r']
#          '1-nearest neighbour',
#          '5-nearest neighbour',
#          'Decision Tree']
#          'LinearSVC',
#          'SVM with a cubic kernel',
#          'SVM with an RBF kernel']

# Select Number of Features
k = 1000

# Chi-square
x2 = SelectKBest(chi2, k=k)
x2.fit(X_train_ingr,y_train)
X_train_ingr_x2 = x2.transform(X_train_ingr)
X_test_ingr_x2 = x2.transform(X_test_ingr)

# Mutual Information 
mi = SelectKBest(score_func=mutual_info_classif, k=k)
mi.fit(X_train_ingr,y_train)
X_train_ingr_mi = mi.transform(X_train_ingr)
X_test_ingr_mi = mi.transform(X_test_ingr)

# Chi-square -> (X_train_ingr_x2, X_test_ingr_x2)
# Mutual Information -> (X_train_ingr_mi, X_test_ingr_mi)

# Fit the model and test the model 
Xs = [(X_train_ingr, X_test_ingr), (X_train_ingr_x2, X_test_ingr_x2) , (X_train_ingr_mi, X_test_ingr_mi)]
X_names = ['complete', 'x2', 'mi']
for title, model in zip(titles, models):
    print('\n',title, '(with k=',k,'features):')
    for X_name, X in zip(X_names, Xs):
        X_train_t, X_test_t = X
        # convert the variable into a matrix and train it with the selected model 
        model.fit(X_train_t.todense(), y_train)
        acc = model.score(X_train_t.todense(), y_train)
        print(X_name, 'acc',  acc)


 GNB (with k= 1000 features):
complete acc 0.189275
x2 acc 0.51075
mi acc 0.23255

 MNB (with k= 1000 features):
complete acc 0.638575
x2 acc 0.627375
mi acc 0.62705

 one-r (with k= 1000 features):
complete acc 0.537825
x2 acc 0.537825
mi acc 0.537825


Here, we could observe that Multinomial Naive Bayes is the best model and Mutual Information is the best feature extraction method for the feature ingredients in the training set

# Joining Training Features for CountVec

In [74]:
# Create a sparse matrix for name
mi_matrix = X_train_name_mi.todense()
# Convert matrix to list
mi_list = mi_matrix.tolist()
# Convert list to dataframe
mi_df = pd.DataFrame(mi_list)
mi_df.shape

(40000, 1000)

In [75]:
# Save Bag of Words of name in training set
# mi_df.to_csv('mi_train_name_countvec.csv', index = False)

In [76]:
# Join the name features to the training set
join_train_df = X_train.join(mi_df)
join_train_df.shape

(40000, 1002)

In [77]:
mi_matrix = X_train_step_mi.todense()
mi_list = mi_matrix.tolist()

mi_step_df = pd.DataFrame(mi_list)

mi_step_df = mi_step_df.add_suffix('step')
mi_step_df.shape

(40000, 1000)

In [78]:
# Save Bag of Words of steps in training set
# mi_step_df.to_csv('mi_train_step_countvec.csv', index = False)

In [79]:
join_train_df = join_train_df.join(mi_step_df)
join_train_df.shape

(40000, 2002)

In [80]:
mi_matrix = X_train_ingr_mi.todense()
mi_list = mi_matrix.tolist()

mi_ingr_df = pd.DataFrame(mi_list)

mi_ingr_df = mi_ingr_df.add_suffix('ingr')

mi_ingr_df.shape

(40000, 1000)

In [81]:
# Save Bag of Words of steps in training set
# mi_ingr_df.to_csv('mi_train_ingr_countvec.csv', index = False)

In [82]:
join_train_df = join_train_df.join(mi_ingr_df)
join_train_df.shape

(40000, 3002)

In [83]:
# Save All Bag of Words in training set
# join_train_df.to_csv('join_train_countvec.csv', index = False)

# Joining Test Features For CountVec

In [84]:
mi_matrix = X_test_name_mi.todense()
mi_list = mi_matrix.tolist()

mi_name_df = pd.DataFrame(mi_list)

In [85]:
# Save Bag of Words of name in testing set
# mi_name_df.to_csv('mi_test_name_countvec.csv', index = False)

In [86]:
join_test_df = X_test.join(mi_name_df)
join_test_df.shape

(10000, 1002)

In [87]:
# MI for step
mi_matrix = X_test_step_mi.todense()
mi_list = mi_matrix.tolist()

mi_step_df = pd.DataFrame(mi_list)
mi_step_df = mi_step_df.add_suffix('step')

In [88]:
# Save Bag of Words of steps in testing set
# mi_step_df.to_csv('mi_test_step_countvec.csv', index = False)

In [89]:
join_test_df = join_test_df.join(mi_step_df)
join_test_df.shape

(10000, 2002)

In [90]:
# MI for ingr
mi_matrix = X_test_ingr_mi.todense()
mi_list = mi_matrix.tolist()

mi_ingr_df = pd.DataFrame(mi_list)
mi_ingr_df = mi_ingr_df.add_suffix('ingr')

In [91]:
# Save Bag of Words of ingr in testing set
# mi_ingr_df.to_csv('mi_test_ingr_countvec.csv', index = False)

In [92]:
join_test_df = join_test_df.join(mi_ingr_df)
join_test_df.shape

(10000, 3002)

In [93]:
# Save All Bag of Words in testing set
# join_test_df.to_csv('join_test_countvec.csv', index = False)

In [46]:
'''
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

models = [GaussianNB(),
          MultinomialNB(),
          DecisionTreeClassifier(max_depth=1),
          #KNeighborsClassifier(),
          LogisticRegression()]
titles = ['GNB',
          'MNB',
          'one-r',
          #'KNN',
          'Logistic Regression']

for title, model in zip(titles, models):
    model.fit(join_train_df,y_train)
    acc = model.score(join_train_df, y_train)
    print(title, "Accuracy:",acc)
''''

SyntaxError: EOL while scanning string literal (<ipython-input-46-d7201a8039fa>, line 21)

## Evaluation - Holdout or Cross Validation

### Hold out Strategy

In [47]:
# Split the training set into train and test set
from sklearn.model_selection import train_test_split
X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(join_train_df, y_train, test_size=0.33, random_state=88)

In [48]:
# Logistic Regression 
from sklearn.linear_model import LogisticRegression
lgr = LogisticRegression()
lgr.fit(X_train_split,y_train_split)
acc = lgr.score(X_test_split, y_test_split)
print("Accuracy:",acc)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Accuracy: 0.7999242424242424


### Cross Validation Strategy

In [49]:
# Logistic Regression 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
lgr = LogisticRegression()
lgr.fit(join_train_df, y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [50]:
acc = np.mean(cross_val_score(lgr, join_train_df, y_train, cv=5))
print("Accuracy:",acc)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Accuracy: 0.79975


Both Holdout and Cross-validation get similar accuracy score. However, holdout is significantly faster than cross validation and for this reason, holdout strategy will be chosen 

## Model Selection 

In [52]:
# Split the training set into train and test set
from sklearn.model_selection import train_test_split
X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(join_train_df, y_train, test_size=0.2,stratify=y_train,random_state=88)

In [54]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train_split, y_train_split)
acc = gnb.score(X_test_split, y_test_split)
print("Accuracy:",acc)

Accuracy: 0.364625


In [55]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
mnb.fit(X_train_split, y_train_split)
acc = mnb.score(X_test_split, y_test_split)
print("Accuracy:",acc)

Accuracy: 0.706125


In [56]:
from sklearn.tree import DecisionTreeClassifier
one_r =DecisionTreeClassifier(max_depth=1)
one_r.fit(X_train_split, y_train_split)
acc = one_r.score(X_test_split, y_test_split)
print("Accuracy:",acc)

Accuracy: 0.641125


In [57]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(X_train_split, y_train_split)
acc = dt.score(X_test_split, y_test_split)
print("Accuracy:",acc)

Accuracy: 0.732375


# Doc2Vec

In [62]:
# Load Doc2Vec Dataset 
train_name_doc2vec100 = pd.read_csv("/Users/hesterlim/COMP30027_2021_Project2_datasets/recipe_text_features_doc2vec100/train_name_doc2vec100.csv", header=None)
test_name_doc2vec100 = pd.read_csv("/Users/hesterlim/COMP30027_2021_Project2_datasets/recipe_text_features_doc2vec100/test_name_doc2vec100.csv", header=None)
train_steps_doc2vec100 = pd.read_csv("/Users/hesterlim/COMP30027_2021_Project2_datasets/recipe_text_features_doc2vec100/train_steps_doc2vec100.csv", header=None)
test_steps_doc2vec100 = pd.read_csv("/Users/hesterlim/COMP30027_2021_Project2_datasets/recipe_text_features_doc2vec100/test_steps_doc2vec100.csv", header=None)
train_ingr_doc2vec100 = pd.read_csv("/Users/hesterlim/COMP30027_2021_Project2_datasets/recipe_text_features_doc2vec100/train_ingr_doc2vec100.csv", header=None)
test_ingr_doc2vec100 = pd.read_csv("/Users/hesterlim/COMP30027_2021_Project2_datasets/recipe_text_features_doc2vec100/test_ingr_doc2vec100.csv", header=None)

In [63]:
print(train_name_doc2vec100.shape, train_steps_doc2vec100.shape, X_train.shape)

(40000, 100) (40000, 100) (40000, 2)


In [64]:
# Load Doc2Vec Dataset 
train_name_doc2vec50 = pd.read_csv("/Users/hesterlim/COMP30027_2021_Project2_datasets/recipe_text_features_doc2vec50/train_name_doc2vec50.csv", header=None)
test_name_doc2vec50 = pd.read_csv("/Users/hesterlim/COMP30027_2021_Project2_datasets/recipe_text_features_doc2vec50/test_name_doc2vec50.csv", header=None)
train_steps_doc2vec50 = pd.read_csv("/Users/hesterlim/COMP30027_2021_Project2_datasets/recipe_text_features_doc2vec50/train_steps_doc2vec50.csv", header=None)
test_steps_doc2vec50 = pd.read_csv("/Users/hesterlim/COMP30027_2021_Project2_datasets/recipe_text_features_doc2vec50/test_steps_doc2vec50.csv", header=None)
train_ingr_doc2vec50 = pd.read_csv("/Users/hesterlim/COMP30027_2021_Project2_datasets/recipe_text_features_doc2vec50/train_ingr_doc2vec50.csv", header=None)
test_ingr_doc2vec50 = pd.read_csv("/Users/hesterlim/COMP30027_2021_Project2_datasets/recipe_text_features_doc2vec50/test_ingr_doc2vec50.csv", header=None)

In [65]:
print(train_name_doc2vec50.shape, train_steps_doc2vec50.shape, X_train.shape)

(40000, 50) (40000, 50) (40000, 2)


In [66]:
# Joining Training Features for Doc2Vec100
train_name_doc2vec100 = train_name_doc2vec100.add_suffix('name_doc2vec100')
join_train_doc2vec100 = train_name_doc2vec100
train_steps_doc2vec100 = train_steps_doc2vec100.add_suffix('step_doc2vec100')
join_train_doc2vec100 = join_train_doc2vec100.join(train_steps_doc2vec100)
train_ingr_doc2vec100 = train_ingr_doc2vec100.add_suffix('ingr_doc2vec100')
join_train_doc2vec100 = join_train_doc2vec100.join(train_ingr_doc2vec100)
join_train_doc2vec100.shape

(40000, 300)

In [67]:
# Joining Testing Features for Doc2Vec100
test_name_doc2vec100 = test_name_doc2vec100.add_suffix('name_doc2vec100')
join_test_doc2vec100 = test_name_doc2vec100
test_steps_doc2vec100 = test_steps_doc2vec100.add_suffix('step_doc2vec100')
join_test_doc2vec100 = join_test_doc2vec100.join(test_steps_doc2vec100)
test_ingr_doc2vec100 = test_ingr_doc2vec100.add_suffix('ingr_doc2vec100')
join_test_doc2vec100 = join_test_doc2vec100.join(test_ingr_doc2vec100)
join_test_doc2vec100.shape

(10000, 300)

In [68]:
# Joining Training Features for Doc2Vec50
train_name_doc2vec50 = train_name_doc2vec50.add_suffix('name_doc2vec50')
join_train_doc2vec50 = train_name_doc2vec50
train_steps_doc2vec50 = train_steps_doc2vec50.add_suffix('step_doc2vec50')
join_train_doc2vec50 = join_train_doc2vec50.join(train_steps_doc2vec50)
train_ingr_doc2vec50 = train_ingr_doc2vec50.add_suffix('ingr_doc2vec50')
join_train_doc2vec50 = join_train_doc2vec50.join(train_ingr_doc2vec50)
join_train_doc2vec50.shape

(40000, 150)

In [69]:
# Joining Testing Features for Doc2Vec100
test_name_doc2vec50 = test_name_doc2vec50.add_suffix('name_doc2vec50')
join_test_doc2vec50 = test_name_doc2vec50
test_steps_doc2vec50 = test_steps_doc2vec50.add_suffix('step_doc2vec50')
join_test_doc2vec50 = join_test_doc2vec50.join(test_steps_doc2vec50)
test_ingr_doc2vec50 = test_ingr_doc2vec50.add_suffix('ingr_doc2vec50')
join_test_doc2vec50 = join_test_doc2vec50.join(test_ingr_doc2vec50)
join_test_doc2vec50.shape

(10000, 150)

In [95]:
# Join all training features
join_train_all = join_train_df.join(join_train_doc2vec100)
join_train_all = join_train_all.join(join_train_doc2vec50)
print(join_train_all.shape)

(40000, 3452)


In [96]:
# Join all testing features
join_test_all = join_test_df.join(join_test_doc2vec100)
join_test_all = join_test_all.join(join_test_doc2vec50)
print(join_test_all.shape)

(10000, 3452)


# Model Training

In [98]:
# Split the training set into train and test set
from sklearn.model_selection import train_test_split
X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(join_train_all, y_train, test_size=0.2,stratify=y_train,random_state=88)

In [139]:
# Logistic Regression 
from sklearn.linear_model import LogisticRegression
lgr = LogisticRegression()
lgr.fit(X_train_split,y_train_split)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [100]:
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

acc = lgr.score(X_test_split, y_test_split)
print("Accuracy:",acc)
#print("f1 score:", f1_score(y_train_split, ))

Accuracy: 0.786875


In [108]:
# Logistic Regression without validation set
from sklearn.linear_model import LogisticRegression
lgr = LogisticRegression()
lgr.fit(join_train_all,y_train)

acc = lgr.score(join_train_all, y_train)
print("Accuracy:",acc)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

# Model Tuning

First, binary logistic regression requires the dependent variable to be binary and ordinal logistic regression requires the dependent variable to be ordinal.

Second, logistic regression requires the observations to be independent of each other.  In other words, the observations should not come from repeated measurements or matched data.

Third, logistic regression requires there to be little or no multicollinearity among the independent variables.  This means that the independent variables should not be too highly correlated with each other.

Fourth, logistic regression assumes linearity of independent variables and log odds.  although this analysis does not require the dependent and independent variables to be related linearly, it requires that the independent variables are linearly related to the log odds.

Finally, logistic regression typically requires a large sample size.  A general guideline is that you need at minimum of 10 cases with the least frequent outcome for each independent variable in your model. For example, if you have 5 independent variables and the expected probability of your least frequent outcome is .10, then you would need a minimum sample size of 500 (10*5 / .10).

In [101]:
# Check distribution
y_train_imbalance = y_train.astype(int)
y_train_imbalance.value_counts()/y_train.shape[0]

2    0.506150
1    0.442625
3    0.051225
Name: duration_label, dtype: float64

Class weight for 1 is 0.44, Class weight for 2 is 0.506 and Class Weight for 3 is 0.05. And there is a significant class imbalnce in Duration Label 3

In [103]:
# Logistic Regression 
from sklearn.linear_model import LogisticRegression
lg2 = LogisticRegression(class_weight='balanced') # inverse class weighting to reduce bias
lg2.fit(X_train_split,y_train_split)

acc = lg2.score(X_test_split, y_test_split)
print("Accuracy:",acc)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [110]:
# Logistic Regression without validation set
from sklearn.linear_model import LogisticRegression
lg2 = LogisticRegression(class_weight='balanced')
lg2.fit(join_train_all, y_train)

#acc = lg2.score(join_train_all, y_train)
#print("Accuracy:",acc)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Accuracy: 0.8192


## Scaling 

In [128]:
# Normalizing features - binary logistic regression requires the dependent variable to be binary and ordinal logistic regression requires the dependent variable to be ordinal.
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(X_train_split)
X_train_scaled = scaler.transform(X_train_split)
X_test_scaled = scaler.transform(X_test_split)

In [129]:
# Does Scaling + fix class imbalance + use a faster solve improve performance?
from sklearn.linear_model import LogisticRegression
lg2 = LogisticRegression(class_weight='balanced', solver = "sag") # inverse class weighting to reduce bias, solve faster
lg2.fit(X_train_scaled,y_train_split)

acc = lg2.score(X_test_scaled , y_test_split)
print("Accuracy:",acc)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Accuracy: 0.770625


In [130]:
# Does Scaling improve performance?  
from sklearn.linear_model import LogisticRegression
lgr = LogisticRegression()
lgr.fit(X_train_scaled,y_train_split)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [131]:
acc = lgr.score(X_test_scaled , y_test_split)
print("Accuracy:",acc)

Accuracy: 0.771125


Scaling does not seem to improve the performance 

## PCA - Reduce Dimension

In [133]:
# Third, logistic regression requires there to be little or no multicollinearity among the independent variables. This means that the independent variables should not be too highly correlated with each other.
# PCA
from sklearn.decomposition import PCA

# Create a PCA that will retain 99% of variance 
pca = PCA(n_components=0.99, whiten = True)

# Conduct PCA
X_train_pca = pca.fit_transform(X_train_scaled)

# Show results 
print("Original number of features:", X_train_scaled.shape[1])
print("Reduced number of features:", X_train_pca.shape[1])

Original number of features: 3452
Reduced number of features: 2984


In [135]:
# Conduct PCA from fitted set
X_test_pca = pca.transform(X_test_scaled)

In [136]:
# Does PCA improve performance? 
from sklearn.linear_model import LogisticRegression
lgr = LogisticRegression()
lgr.fit(X_train_pca,y_train_split)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [137]:
acc = lgr.score(X_test_pca , y_test_split)
print("Accuracy:",acc)

Accuracy: 0.769875


In [138]:
# Does Scaling + fix class imbalance + use a faster solve improve performance + reduce correlation? 
from sklearn.linear_model import LogisticRegression
lg2 = LogisticRegression(class_weight='balanced', solver = "sag") # inverse class weighting to reduce bias, solve faster
lg2.fit(X_train_pca,y_train_split)

acc = lg2.score(X_test_pca, y_test_split)
print("Accuracy:",acc)

Accuracy: 0.76725


/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


# Ensemble - Stacking

In [115]:
from sklearn.metrics import accuracy_score

np.random.seed(1)

class StackingClassifier():

    def __init__(self, classifiers, metaclassifier):
        self.classifiers = classifiers
        self.metaclassifier = metaclassifier

    def fit(self, X, y):
        for clf in self.classifiers:
            clf.fit(X, y)
        X_meta = self._predict_base(X)
        self.metaclassifier.fit(X_meta, y)
    
    def _predict_base(self, X):
        yhats = []
        for clf in self.classifiers:
            yhat = clf.predict_proba(X)
            yhats.append(yhat)
        yhats = np.concatenate(yhats, axis=1)
        assert yhats.shape[0] == X.shape[0]
        return yhats
    
    def predict(self, X):
        X_meta = self._predict_base(X)     
        yhat = self.metaclassifier.predict(X_meta)
        return yhat
    def score(self, X, y):
        yhat = self.predict(X)
        return accuracy_score(y, yhat)
    


classifiers = [#DummyClassifier(strategy='most_frequent'),
                LogisticRegression(),
                #KNeighborsClassifier(),
                GaussianNB(),
                #MultinomialNB(),
                DecisionTreeClassifier()]
titles = [#'Zero_R',
          'Logistic Regression',
          #'KNN',
          'Gaussian NB',  
          #'Multinomial NB',
          'Decision Tree']



meta_classifier_lr = LogisticRegression()
stacker_lr = StackingClassifier(classifiers, meta_classifier_lr)

In [118]:
# Run with validation set
stacker_lr.fit(X_train_split,y_train_split)
print('\nStacker Accuracy (Logistic Regression):', stacker_lr.score(X_test_split, y_test_split))

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)



Stacker Accuracy (Logistic Regression): 0.703125


In [119]:
# Run without validation set
stacker_lr.fit(join_train_all, y_train)
print('\nStacker Accuracy (Logistic Regression):', stacker_lr.score(X_test_split, y_test_split))

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)



Stacker Accuracy (Logistic Regression): 1.0


## Prediction

In [111]:
ybar = lgr.predict(join_test_all)

In [112]:
test_id = test_df.index
data = {'id': test_id+1, 'duration_label': ybar}
df = pd.DataFrame(data)
df.to_csv('predict.csv', index=False)

In [53]:
'''
# Neural Network
from keras import models
from keras import layers
from keras.utils.np_utils import to_categorical

from sklearn import preprocessing

X_train_split_neural = X_train_split
X_test_split_neural = X_test_split

#Create Scalar
scaler = preprocessing.StandardScaler()

# Transform the feature
X_train_split_neural = scaler.fit_transform(X_train_split_neural)
X_test_split_neural = scaler.fit_transform(X_test_split_neural)

print(X_train_split_neural[:,0].mean(),X_train_split_neural[:,0].std())

y_train_split_neural = y_train_split - 1
y_test_split_neural = y_test_split - 1

# One-hot encode target vector to create a target matrix
y_train_split_cat = to_categorical(y_train_split_neural)
y_test_split_cat = to_categorical(y_test_split_neural)

print(y_train_split_cat.shape)

# Set the number of featuers we want
number_of_features = 3002

# Start neural network
network = models.Sequential()

# Add an embedding layer
network.add(layers.Dense(units=100, activation="relu", input_shape=(number_of_features,)))

# Add a long short-term memory layer with 128 units
network.add(layers.Dense(units=100, activation = "relu"))

#Add fully connected layer with a softmax activation function
network.add(layers.Dense(units=3, activation="softmax"))

#Compile neural network
network.compile(loss="categorical_crossentropy", #Cross-entropy
               optimizer = "Adam", # Adam Optimization
               metrics=["accuracy"]) # Accuracy performance metric

# Train neural network
history = network.fit(X_train_split_neural, #Features
                     y_train_split_cat, #Target
                     epochs=10, #one epoch
                     verbose=1, #No output
                     batch_size=100, #Number of observations per batch
                     validation_data=(X_test_split_neural, y_test_split_cat)) # test data
                     
# Change the Test Dataset for prediction
join_test_neural = join_test_df
# Transform the feature 
join_test_neural = scaler.fit_transform(join_test_neural)
print(join_test_neural[:,0].mean(),join_test_neural[:,0].std())

# Prediction 
ybar = network.predict(join_test_neural) 
ybar

# Post process prediction 
y_bar_list = []
for row in ybar:
    max_value = max(row)
    for column in range(len(row)):
        if row[column] == max_value:
            max_index = column + 1
            y_bar_list.append(max_index)
            
ybar = lgr.predict(join_test_all)
'''

'\n# Neural Network\nfrom keras import models\nfrom keras import layers\nfrom keras.utils.np_utils import to_categorical\n\nfrom sklearn import preprocessing\n\nX_train_split_neural = X_train_split\nX_test_split_neural = X_test_split\n\n#Create Scalar\nscaler = preprocessing.StandardScaler()\n\n# Transform the feature\nX_train_split_neural = scaler.fit_transform(X_train_split_neural)\nX_test_split_neural = scaler.fit_transform(X_test_split_neural)\n\nprint(X_train_split_neural[:,0].mean(),X_train_split_neural[:,0].std())\n\ny_train_split_neural = y_train_split - 1\ny_test_split_neural = y_test_split - 1\n\n# One-hot encode target vector to create a target matrix\ny_train_split_cat = to_categorical(y_train_split_neural)\ny_test_split_cat = to_categorical(y_test_split_neural)\n\nprint(y_train_split_cat.shape)\n\n# Set the number of featuers we want\nnumber_of_features = 3002\n\n# Start neural network\nnetwork = models.Sequential()\n\n# Add an embedding layer\nnetwork.add(layers.Dense(uni